How to use this facial recognition software:

Step 1: Run the "Installs and Imports" code cell. This will install and import all of the necessary tools for the program.


Step 2: If you would like to add a face to the accepted faces, run the "Add a face to accepted faces" code cell. This will open the webcam on the user device. When you are ready to take the photo, click the 'Capture' button.


Step 3: If you would like to remove a face from the accepted faces, run the "Remove a face from accepted faces" code cell. After asking you for the name of the face you would like to remove, this will remove the indicated face from the accepted faces.


Step 4: To verify a user's face, run the "Verify current user's face" code cell. This will open the webcam and you will have 5 seconds to position yourself before the photo is taken. The program will then modify your picture to indicate if the face identified in the picture is a member of the accepted faces.

Installs and Imports:

In [ ]:
!pip install keras_facenet
from IPython.display import display, Javascript
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
import cv2 as cv
from google.colab.patches import cv2_imshow
from keras_facenet import FaceNet
from IPython.display import clear_output
import os
import matplotlib as plt
import pickle
from PIL import Image

# Load the pre-trained FaceNet model
facenet_model = FaceNet()

#Ensure correct directory is configured
directory = '/content/drive/MyDrive/CS362V/data/faces'
if not os.path.exists(directory):
  os.makedirs(directory)

Add a face to accepted faces:

In [ ]:
# Define the function to take a photo
def take_photo_and_process(filename='photo.jpg', quality=0.8):
    # JavaScript code to take a photo
    js = Javascript('''
        async function takePhoto(quality) {
          const div = document.createElement('div');
          const capture = document.createElement('button');
          capture.textContent = 'Capture';
          div.appendChild(capture);

          const video = document.createElement('video');
          video.style.display = 'block';
          const stream = await navigator.mediaDevices.getUserMedia({video: true});

          document.body.appendChild(div);
          div.appendChild(video);
          video.srcObject = stream;
          await video.play();

          // Resize the output to fit the video element.
          google.colab.output.setIframeHeight(document.documentElement.scrollHeight, true);

          // Wait for Capture to be clicked.
          await new Promise((resolve) => capture.onclick = resolve);

          const canvas = document.createElement('canvas');
          canvas.width = video.videoWidth;
          canvas.height = video.videoHeight;
          canvas.getContext('2d').drawImage(video, 0, 0);
          stream.getVideoTracks()[0].stop();
          div.remove();
          return canvas.toDataURL('image/jpeg', quality);
        }
    ''')

    # Display the JavaScript code and capture the photo
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])

    # Save the photo to a file
    with open(filename, 'wb') as f:
        f.write(binary)

# Define the function to process the photo using OpenCV
def processPhoto(filename):

    full_path = os.path.join('/content/drive/MyDrive/CS362V/data/faces', filename)

    # Load the cascade
    face_cascade_name = '/content/drive/MyDrive/opencv/data/haarcascades/haarcascade_frontalface_alt.xml'
    face_cascade = cv.CascadeClassifier()

    if not face_cascade.load(cv.samples.findFile(face_cascade_name)):
        print('--(!)Error loading face cascade')
        return

    # Read the image using OpenCV
    frame = cv.imread(full_path)

    if frame is None:
      print(f'--(!)Error reading image: {filename}')
      return

    # Convert the image to grayscale
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    frame_gray = cv.equalizeHist(frame_gray)

    # Detect faces
    faces = face_cascade.detectMultiScale(frame_gray, 1.1, 4)

    if len(faces) > 0:
      x1, y1, width, height = faces[0]
    else:
      x1, y1, width, height = 1, 1, 10, 10

    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height

    gbr = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    gbr = Image.fromarray(gbr)
    gbr_array = np.asarray(gbr)

    face = gbr_array[y1:y2, x1:x2]

    face = Image.fromarray(face)
    face = face.resize((160,160))
    face = np.asarray(face)

    face = np.expand_dims(face, axis=0)
    signature = facenet_model.embeddings(face)

    return {os.path.splitext(filename)[0] : signature}

name = str(input('Enter first name of face to add: '))
take_photo_and_process(filename='/content/drive/MyDrive/CS362V/data/faces/%s.jpg' % (name))

try:
  with open("/content/drive/MyDrive/CS362V/data/data.pkl", "rb") as data_file:
    existing_data = pickle.load(data_file)
except FileNotFoundError:
    existing_data = {}

existing_data.update(processPhoto("%s.jpg" % (name)))

with open("/content/drive/MyDrive/CS362V/data/data.pkl", 'wb') as data_file:
    pickle.dump(existing_data, data_file)

Remove a face from accepted faces:

In [ ]:
# Define the function to process the photo using OpenCV
def process_photo(filename):

    full_path = os.path.join('/content/drive/MyDrive/CS362V/data/faces', filename)

    # Load the cascade
    face_cascade_name = '/content/drive/MyDrive/opencv/data/haarcascades/haarcascade_frontalface_alt.xml'
    face_cascade = cv.CascadeClassifier()

    if not face_cascade.load(cv.samples.findFile(face_cascade_name)):
        print('--(!)Error loading face cascade')
        return

    # Read the image using OpenCV
    frame = cv.imread(full_path)

    if frame is None:
      print(f'--(!)Error reading image: {filename}')
      return

    # Convert the image to grayscale
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    frame_gray = cv.equalizeHist(frame_gray)

    # Detect faces
    faces = face_cascade.detectMultiScale(frame_gray, 1.1, 4)

    if len(faces) > 0:
      x1, y1, width, height = faces[0]
    else:
      x1, y1, width, height = 1, 1, 10, 10

    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height

    gbr = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    gbr = Image.fromarray(gbr)
    gbr_array = np.asarray(gbr)

    face = gbr_array[y1:y2, x1:x2]

    face = Image.fromarray(face)
    face = face.resize((160,160))
    face = np.asarray(face)

    face = np.expand_dims(face, axis=0)
    signature = facenet_model.embeddings(face)

    database[os.path.splitext(filename)[0]] = signature

rm_name = str(input("Enter the first name of the face you would like to remove: "))
file_path = '/content/drive/MyDrive/CS362V/data/faces/%s.jpg' % (rm_name)

if os.path.exists(file_path):
    try:
        os.remove(file_path)
        print(f"'{rm_name}' has been successfully removed.")
    except OSError as e:
        print(f"Error: {e}")
else:
    print(f"'{rm_name}' does not exist.")

database = {}
for filename in os.listdir('/content/drive/MyDrive/CS362V/data/faces'):
  process_photo(filename)

with open("/content/drive/MyDrive/CS362V/data/data.pkl", "wb") as data_file:
  pickle.dump(database, data_file)

Verify current user's face:

In [ ]:
with open("/content/drive/MyDrive/CS362V/data/data.pkl", "rb") as data_file:
  database = pickle.load(data_file)

def take_photo_and_process(filename='photo.jpg', quality=0.8):
    # JavaScript code to take a photo after 3 seconds
    js = Javascript('''
        async function takePhoto(quality) {
            const video = document.createElement('video');
            video.style.display = 'block';
            const stream = await navigator.mediaDevices.getUserMedia({video: true});

            document.body.appendChild(video);
            video.srcObject = stream;
            await video.play();

            // Wait for 3 seconds before capturing the photo
            await new Promise((resolve) => setTimeout(resolve, 5000));

            // Capture the photo
            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            canvas.getContext('2d').drawImage(video, 0, 0);

            // Stop the stream and remove video element
            stream.getVideoTracks()[0].stop();
            video.remove();

            // Return the photo data URL
            return canvas.toDataURL('image/jpeg', quality);
        }
    ''')

    # Display the JavaScript code and capture the photo
    display(js)
    data = eval_js('takePhoto({})'.format(quality))
    binary = b64decode(data.split(',')[1])

    # Save the photo to a file
    with open(filename, 'wb') as f:
        f.write(binary)

    # Process the photo using OpenCV
    process_photo(filename)

# Define the function to process the photo using OpenCV
def process_photo(filename):

    # Load the cascade
    face_cascade_name = '/content/drive/MyDrive/opencv/data/haarcascades/haarcascade_frontalface_alt.xml'
    face_cascade = cv.CascadeClassifier()

    if not face_cascade.load(cv.samples.findFile(face_cascade_name)):
        print('--(!)Error loading face cascade')
        return

    # Read the image using OpenCV
    frame = cv.imread(filename)

    # Convert the image to grayscale
    frame_gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
    frame_gray = cv.equalizeHist(frame_gray)

    # Detect faces
    faces = face_cascade.detectMultiScale(frame_gray, 1.1, 4)

    if len(faces) > 0:
      x1, y1, width, height = faces[0]
    else:
      x1, y1, width, height = 1, 1, 10, 10

    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height

    gbr = cv.cvtColor(frame, cv.COLOR_BGR2RGB)
    gbr = Image.fromarray(gbr)
    gbr_array = np.asarray(gbr)

    face = gbr_array[y1:y2, x1:x2]

    face = Image.fromarray(face)
    face = face.resize((160,160))
    face = np.asarray(face)

    face = np.expand_dims(face, axis=0)
    signature = facenet_model.embeddings(face)

    min_dist = 100
    identity = ""
    for key, value in database.items():
      dist = np.linalg.norm(value - signature)
      if dist < min_dist:
        min_dist = dist
        identity = key

    # Determine whether the face is recognized
    recognition_threshold = 1
    print(min_dist, recognition_threshold)
    if min_dist <= recognition_threshold:
        welcome_text = f'Welcome, {identity}.'
        text_color = (0, 255, 0)
    else:
        # If the minimum distance exceeds the threshold, the face is not recognized
        welcome_text = 'Not Verified.'
        text_color = (0, 0, 255)

    cv.putText(frame, welcome_text, (50, 50), cv.FONT_HERSHEY_SIMPLEX, 1, text_color, 2)
    cv.rectangle(frame, (x1, y1), (x2, y2), (255, 0, 205), 2)

    cv2_imshow(frame)

take_photo_and_process()